In [1]:
import re
from selenium import webdriver
from time import sleep
browser = webdriver.Chrome()

In [2]:
with open('memo.txt', 'r', encoding='utf8') as f:
    name_ls = []
    for name in f.read().splitlines():
        name_ls.append(name)

name_en_ls = []
browser.get('https://www.hareruyamtg.com/ja/')
for name in name_ls:
    browser.find_element_by_xpath('//*[@id="leftmenu_search_goods_name"]').clear()
    sleep(1)
    browser.find_element_by_xpath('//*[@id="leftmenu_search_goods_name"]').send_keys(name)
    browser.find_element_by_xpath('//*[@id="leftmenu_search_goods_submit"]').click()
    elem_box = browser.find_element_by_class_name('itemList')

    name = elem_box.find_element_by_class_name('itemName').text
    name_from = name.find('/') + 1
    name_end = name.find('》')
    query_pre = '{"bool": {"must": [{"match": {"name.keyword": "'
    query_suf = '"}}]}},'
    name = query_pre + name[name_from:name_end] + query_suf
    name_en_ls.append(name)
    sleep(2)

# 最後の句は最後の,を削除
tmp_name = name_en_ls[-1]
name_en_ls[-1] = tmp_name[:-1]

with open('memo_post_query.txt', 'w', encoding='utf8') as f:
    name_en = "\n".join(name_en_ls)
    post_query_pre = 'POST /mtg/_update_by_query?conflicts=proceed&pretty\n{"query": {"bool": {"should": ['
    post_query_suf = ']}},"script": {"inline": """ctx._source.legalities.historic= "Legal";"""}}'
    f.write(post_query_pre + name_en + post_query_suf)

with open('memo_get_query.txt', 'w', encoding='utf8') as f:
    name_en = "\n".join(name_en_ls)
    get_query_pre = 'GET /mtg/_search\n{"_source": ["_id","name","legalities"],"size": 1000, "query": {"bool": {"should": ['
    get_query_suf = ']}}}'
    f.write(get_query_pre + name_en + get_query_suf)